<a href="https://colab.research.google.com/github/Tenura2001/TinyML/blob/main/TensorFlowLite_Converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Colab, we will explore the ***TensorFlow Lite Converter*** to create a TFLite model from the original TensorFlow model.

First import the following Libraries-

1.   tensorflow - open source library to develop and train ML models
2.   numpy - scientific computing in python
3.   keras - deep learning framework buit on top of TensorFlow





In [27]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# 1. Define TensorFlow Model
We consider a single-layer neural network for a Linear Regression Task - modeling Y = 2X+3. We define the optimizer to be 'Stochastic Gradient Descent' and the loss fun to be 'Mean Squared Error'.

In [28]:
from tensorflow.keras.layers import Input

my_layer = Dense(units=1)
model = Sequential([Input(shape=[1]), my_layer])

model.compile(optimizer='sgd', loss='mean_squared_error')

xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([1.0, 3.0, 5.0, 7.0, 9.0, 11.0], dtype=float)

# Train Model

In [29]:
model.fit(xs, ys, epochs=500)

Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step - loss: 38.9908
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 31.2190
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 25.0933
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 20.2629
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 16.4520
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 13.4432
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 11.0658
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 9.1853
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 7.6960
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 6.5147
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - loss: 5.5758
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 4.8279
Epoch 13/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 4.2304
Epoch 14/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 3.7514
Epoch 15/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 3.3659
Epoch 16/5

# Predict X = 10.0 and inspect learned weights

In [30]:
print(model.predict(np.array([10.0])))
print("Here are the parameters that the model learned:")
print(my_layer.get_weights())

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
[[23.027603]]
Here are the parameters that the model learned:
[array([[2.004001]], dtype=float32), array([2.9875956], dtype=float32)]


# Save Model
We save our trained model to a specified path.This is done using the `save_model_save()` API that exports a TensorFlow model object to Save Model format.

In [31]:
export_dir = 'saved_TF_model/model1'
tf.saved_model.save(model, export_dir)

# 2. Convert TF Model to TFLite Model

We use the `tf.lite.TFLiteConverter.from_saved_model()` API to load the saved TF model and then use the `convert()` method to transform into the TFLite model.

In [32]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

Saved artifact at '/tmp/tmpynsyal5w'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1), dtype=tf.float32, name='keras_tensor_4')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  136738582130384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136738582132112: TensorSpec(shape=(), dtype=tf.resource, name=None)


# Storing TFLite model onto disk
We import the following:
1. pathlib - Obejct-oriented filesystem paths

We write the TFLite model(model.tflite)to the disk - this file encapsulates the model and its saved weights. This is the compressed model that we deploy on edge devices!

We first create the desired path using the `pathlib.Path()` API.We then write the contents from our converted tflite model to this path using the `tflite_model_file.write_bytes()` API.

In [33]:
import pathlib
tflite_model_file = pathlib.Path('model.tflite')
print("Size of TFLite Model in bytes")
tflite_model_file.write_bytes(tflite_model)

Size of TFLite Model in bytes


1116

# Model Inference using TensorFlow Interpreter
Now that we have a TFLite model, how do we test it without actually needing an edge device?

We dont need a RPi,smartphone or an embedded system! TensorFlow lite provides an interface called **Interpreter** that allows us to load the TF lite model and run inference in Python.

We use the tf.lite.Interpreter() API to load the tflite model to the Interpreter and we extract the input and output tensor detailes to run inference. The input and output details help us define important details such as the shape of our input for inferece and expected output after inference.It also defines the index of the interpreter assigned for the input and output tensors.

In [34]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'serving_default_keras_tensor_4:0', 'index': 0, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall_1:0', 'index': 3, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


We then format our value to be predicted(10.) as a tensor according to how the model accepts input, run inference, and read the result on the output tensor and store it in a variable (here,tflite_result)

In [35]:
to_predict = np.array([[10.]],dtype=np.float32)
print("value to predict:",to_predict)

interpreter.set_tensor(input_details[0]['index'], to_predict)

interpreter.invoke()

tflite_result = interpreter.get_tensor(output_details[0]['index'])
print("predicted value:",tflite_result)

value to predict: [[10.]]
predicted value: [[23.027603]]


In [36]:
# Inspect the TFLite model
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

print("Tensor Details:")
for tensor_detail in interpreter.get_tensor_details():
    print(tensor_detail)

Tensor Details:
{'name': 'serving_default_keras_tensor_4:0', 'index': 0, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
{'name': 'sequential_2_1/dense_2_1/Add/ReadVariableOp', 'index': 1, 'shape': array([1], dtype=int32), 'shape_signature': array([1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
{'name': 'sequential_2_1/dense_2_1/MatMul', 'index': 2, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([1, 1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32